# 深層強化学習（DQN）による手順獲得問題

スキナーの箱とも呼ばれる

## ライブラリのインストール

In [9]:
!pip install tf-agents

In [8]:
import tf_agents

ModuleNotFoundError: No module named 'tf_agents'

## ライブラリのインストール

In [2]:
import tensorflow as tf
from tensorflow import keras

from tf_agents.environments import gym_wrapper, py_environment, tf_py_environment
from tf_agents.agents.dqn import dqn_agent
from tf_agents.networks import network
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.policies import policy_saver
from tf_agents.trajectories import time_step as ts
from tf_agents.trajectories import trajectory
from tf_agents.specs import array_spec
from tf_agents.utils import common
from tf_agents.drivers import dynamic_step_driver, dynamic_episode_driver

import numpy as np
import random


ModuleNotFoundError: No module named 'tf_agents'

## シミュレータクラスの設定

In [2]:
class EnvironmentSimulator(py_environment.PyEnvironment):
    def __init__(self):
        super(EnvironmentSimulator, self).__init__()
        self._observation_spec = array_spec.BoundedArraySpec(
            shape=(1,), dtype=np.int32, minimum=0, maximum=1
        )
        self._action_spec = array_spec.BoundedArraySpec(
            shape=(), dtype=np.int32, minimum=0, maximum=1
        )
        self._reset()
    def observation_spec(self):
        return self._observation_spec
    def action_spec(self):
        return self._action_spec
#初期化
    def _reset(self):
        self._state = 0
        return ts.restart(np.array([self._state], dtype=np.int32))
#行動による状態変化
    def _step(self, action):
        reward = 0
        if self._state == 0:#電源OFFの状態
            if action == 0:#電源ボタンを押す
                self._state = 1#電源ON
            else:#行動ボタンを押す
                self._state = 0#電源OFF
        else:#電源ONの状態
            if action == 0:
                self._state = 0
            else:
                self._state = 1
                reward = 1#報酬が得られる
        return ts.transition(np.array([self._state], dtype=np.int32), reward=reward, discount=1)#TF-Agents用の戻り値の生成


## ネットワーククラスの設定

In [3]:
class MyQNetwork(network.Network):
    def __init__(self, observation_spec, action_spec, n_hidden_channels=2, name='QNetwork'):
        super(MyQNetwork, self).__init__(
            input_tensor_spec=observation_spec, 
            state_spec=(), 
            name=name
        )
        n_action = action_spec.maximum - action_spec.minimum + 1
        self.model = keras.Sequential(
        [
            keras.layers.Dense(n_hidden_channels, activation='tanh'),
            keras.layers.Dense(n_hidden_channels, activation='tanh'),
            keras.layers.Dense(n_action),
        ]
        )
    def call(self, observation, step_type=None, network_state=(), training=True):
        actions = self.model(observation, training=training)
        return actions, network_state


## その他の設定

In [4]:
#環境の設定
env_py = EnvironmentSimulator()
env = tf_py_environment.TFPyEnvironment(env_py)
#ネットワークの設定
primary_network = MyQNetwork(env.observation_spec(), env.action_spec())
#エージェントの設定
n_step_update = 1
agent = dqn_agent.DqnAgent(
    env.time_step_spec(),
    env.action_spec(),
    q_network=primary_network,
    optimizer=keras.optimizers.Adam(learning_rate=1e-2, epsilon=1e-2),
    n_step_update=n_step_update,
    epsilon_greedy=1.0,
    target_update_tau=1.0,
    target_update_period=10,
    gamma=0.9,
    td_errors_loss_fn = common.element_wise_squared_loss,
    train_step_counter = tf.Variable(0)
)
agent.initialize()
agent.train = common.function(agent.train)
#行動の設定
policy = agent.collect_policy
#データの保存の設定
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size,
    max_length=10**6
)
dataset = replay_buffer.as_dataset(
    num_parallel_calls=tf.data.experimental.AUTOTUNE,
    sample_batch_size=32,
    num_steps=n_step_update+1
).prefetch(tf.data.experimental.AUTOTUNE)
iterator = iter(dataset)
#事前データの設定
env.reset()
driver = dynamic_step_driver.DynamicStepDriver(
    env, 
    policy, 
    observers=[replay_buffer.add_batch], 
    num_steps = 100,
)
driver.run(maximum_iterations=100)

num_episodes = 100
epsilon = np.linspace(start=1.0, stop=0.0, num=num_episodes+1)#ε-greedy法用
tf_policy_saver = policy_saver.PolicySaver(policy=agent.policy)#ポリシーの保存設定


Instructions for updating:
Use `tf.data.Dataset.counter(...)` instead.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


## 学習

In [5]:
for episode in range(num_episodes+1):
    episode_rewards = 0#報酬の計算用
    episode_average_loss = []#lossの計算用
    policy._epsilon = epsilon[episode]#エピソードに合わせたランダム行動の確率
    time_step = env.reset()#環境の初期化

    for t in range(5):#各エピソード5回の行動
        policy_step = policy.action(time_step)#状態から行動の決定
        next_time_step = env.step(policy_step.action)#行動による状態の遷移

        traj =  trajectory.from_transition(time_step, policy_step, next_time_step)#データの生成
        replay_buffer.add_batch(traj)#データの保存

        experience, _ = next(iterator)#学習用データの呼び出し   
        loss_info = agent.train(experience=experience)#学習

        S = time_step.observation.numpy().tolist()[0]#状態
        A = policy_step.action.numpy().tolist()[0]#行動
        R = next_time_step.reward.numpy().astype('int').tolist()[0]#報酬
        print(S, A, R)
        episode_average_loss.append(loss_info.loss.numpy())#lossの計算
        episode_rewards += R#報酬の合計値の計算

        time_step = next_time_step#次の状態を今の状態に設定

    print(f'Episode:{episode:4.0f}, R:{episode_rewards:3.0f}, AL:{np.mean(episode_average_loss):.4f}, PE:{policy._epsilon:.6f}')

tf_policy_saver.save(export_dir='policy')#ポリシーの保存
env.close()


Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))
[0] 1 0
[0] 0 0
[1] 0 0
[0] 1 0
[0] 0 0
Episode:   0, R:  0, AL:0.9929, PE:1.000000
[0] 1 0
[0] 0 0
[1] 1 1
[1] 0 0
[0] 0 0
Episode:   1, R:  1, AL:0.9971, PE:0.990000
[0] 0 0
[1] 1 1
[1] 0 0
[0] 0 0
[1] 1 1
Episode:   2, R:  2, AL:0.4959, PE:0.980000
[0] 0 0
[1] 1 1
[1] 0 0
[0] 0 0
[1] 0 0
Episode:   3, R:  1, AL:0.2386, PE:0.970000
[0] 1 0
[0] 0 0
[1] 0 0
[0] 0 0
[1] 1 1
Episode:   4, R:  1, AL:0.2975, PE:0.960000
[0] 1 0
[0] 1 0
[0] 0 0
[1] 1 1
[1] 1 1
Episode:   5, R:  2, AL:0.1794, PE:0.950000
[0] 1 0
[0] 0 0
[1] 0 0
[0] 0 0
[1] 0 0
Episode:   6, R:  0, AL:0.3225, PE:0.940000
[0] 1 0
[0] 0 0
[1] 1 1
[1] 0 0
[0] 0 0
Episode:   7, R:  1, AL:0.2040, PE:0.930000
[0] 0 0
[1] 1 1
[1] 0 0
[0] 0 0
[1] 0 0
Episode:   8, R:  1, AL:0.3319, PE:0.920000
[0] 0 0
[1

[1] 1 1
[1] 1 1
[1] 1 1
Episode:  93, R:  4, AL:0.1411, PE:0.070000
[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode:  94, R:  4, AL:0.1150, PE:0.060000
[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode:  95, R:  4, AL:0.0981, PE:0.050000
[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode:  96, R:  4, AL:0.1120, PE:0.040000
[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode:  97, R:  4, AL:0.1031, PE:0.030000
[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode:  98, R:  4, AL:0.1098, PE:0.020000


[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode:  99, R:  4, AL:0.1050, PE:0.010000
[0] 0 0
[1] 1 1
[1] 1 1
[1] 1 1
[1] 1 1
Episode: 100, R:  4, AL:0.1310, PE:0.000000


C:\Users\makino\anaconda3\envs\DS\lib\site-packages\tensorflow\python\saved_model\nested_structure_coder.py:497: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


FailedPreconditionError: policy\variables is not a directory

In [6]:
tf_policy_saver.save(export_dir='policy')#ポリシーの保存


C:\Users\makino\anaconda3\envs\DS\lib\site-packages\tensorflow\python\saved_model\nested_structure_coder.py:497: UserWarning: Encoding a StructuredValue with type tfp.distributions.Deterministic_ACTTypeSpec; loading this StructuredValue will require that this type be imported and registered.
  warnings.warn("Encoding a StructuredValue with type %s; loading this "


FailedPreconditionError: policy\variables is not a directory